In [ ]:
from river.datasets import Insects
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time

In [ ]:
dataset = Insects('abrupt_balanced')


In [ ]:
learner = CluStream(n_macro_clusters=6,max_micro_clusters=60,time_window=300)
test = drift.PageHinkley(30,delta=0.05,threshold=30,alpha= 0.9999,mode='both')



In [ ]:
#Deprecated
def _get_mean_dist(learner):
    ## Calcule la moyenne des max distances entres les micro clusters
    start = time()
    mc_centers = dict2numpy(learner.get_micro_clusters())
    nb_centers = len(mc_centers)
    dist_matrix = np.zeros(shape=(nb_centers,nb_centers))
    
    for i in range(nb_centers):
        for j in range(nb_centers):
            dist_matrix[i][j] = learner._distance(mc_centers[i],mc_centers[j])

    max_dist = np.max(dist_matrix,axis=1)
    max_tmax = np.max(max_dist)
    end = time() - start
    print(end)
    return np.mean(max_dist), max_tmax


In [ ]:
def get_mean_dist(learner):
    ## Calcule la moyenne des max distances entres les centres des micro clusters
    mc_centers = [dict2numpy(x) for x in dict2numpy(learner.get_micro_clusters())]
    max_dist = np.max(distance_matrix(mc_centers,mc_centers,2),axis=1)
    return np.mean(max_dist)

In [ ]:
loop = 0
last_dist = 0
dist = 0
for x,y in dataset:
    if loop <= 200 :
        learner.learn_one(x)
        
    else :
        if loop == 201 :
            print("Done training")
            learner.offline_cluster()

        if loop == 5000:
            break
        
        learner.learn_one(x)
        dist, max = get_mean_dist(learner)
        print(f"{dist} , {max}")
        test.update(dist)
        
        if test.drift_detected : 
            print(f"Drift Detected at instance: {loop} , last dist : {last_dist}, dist : {dist}")
        
            
        
    loop += 1
    last_dist = dist

    